# Assigment 05

Now that we already have the data from previous assgiment, let import the files and start to work with them.

In [1]:
#Import the librarys
import pandas as pd

from tqdm import tqdm
tqdm.pandas(desc="Processing")

In [2]:
data = pd.read_pickle('./data/raw/10-K_Data.pik')

## Removing the html tags

The files we have imported have html tags and it is very dificult to read, lets tale a look

In [3]:
sample = data['corpus'][0]
print(sample[3000:4000])

b'ollapse;text-align:left;"><tr><td colspan="2"></td></tr><tr><td style="width:6%;"></td><td style="width:94%;"></td></tr><tr><td style="vertical-align:top;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:left;font-size:12pt;"><font style="font-family:Wingdings;font-size:12pt;">x</font></div></td><td style="vertical-align:top;padding-left:2px;padding-top:2px;padding-bottom:2px;padding-right:2px;"><div style="text-align:left;font-size:12pt;"><font style="font-family:inherit;font-size:12pt;font-weight:bold;">ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934</font></div></td></tr></table></div></div><div style="line-height:120%;padding-top:12px;text-align:center;font-size:10pt;"><font style="font-family:inherit;font-size:10pt;font-weight:bold;">For the fiscal year ended </font><font style="font-family:inherit;font-size:10pt;color:#000000;font-weight:bold;text-decoration:none;">December&#160;31, 2016</font><fo

In [4]:
from bs4 import BeautifulSoup
import unicodedata
import re

def text_process(text):
    """
    Preprocess Text
    """
    #Remove the  html tags
    soup = BeautifulSoup(text, "html.parser" )
    text = soup.get_text("\n")
    
    # Normalize
    text = unicodedata.normalize("NFKD", text) 
    text = '\n'.join(text.splitlines())

    # Take care of breaklines & whitespaces combinations due to beautifulsoup parsing
    text = re.sub(r'[ ]+\n', '\n', text)
    text = re.sub(r'\n[ ]+', '\n', text)
    text = re.sub(r'\n+', '\n', text)

    # Reformat item headers
    text = text.replace('\n.\n','.\n') # Move Period to beginning

    text = text.replace('\nI\nTEM','\nITEM')
    text = text.replace('\nITEM\n','\nITEM ')
    text = text.replace('\nITEM  ','\nITEM ')

    text = text.replace(':\n','.\n')

    # Math symbols for clearer looks
    text = text.replace('$\n','$')
    text = text.replace('\n%','%')
    
    return text

In [5]:
# Check to make sure its working
sample_clean = data['corpus'].head(1).apply(text_process)
print(list(sample_clean)[0][3000:4000])

egistrant has submitted electronically and posted on its corporate website, if any, every Interactive Data File required to be submitted and posted pursuant to Rule 405 of Regulation S-T during the preceding 12 months (or for such shorter period that the registrant was required to submit and post such files).    Yes
x
No
̈
Indicate by check mark if disclosure of delinquent filers pursuant to Item 405 of Regulation S-K is not contained herein, and will not be contained, to the best of Registrant’s knowledge, in definitive proxy or information statements incorporated by reference in Part III of this Form 10-K or any amendment to this Form 10-K.
x
Indicate by check mark whether the Registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, or a smaller reporting company. See the definitions of “large accelerated filer,” “accelerated filer” and “smaller reporting company” in Rule 12b-2 of the Exchange Act. (Check one).
Large accelerated filer
̈
Accelerated file

In [6]:
data['corpus'] = data['corpus'].progress_apply(text_process)

Processing: 100%|██████████| 100/100 [20:44<00:00,  4.82s/it] 


In [8]:
print(list(data['corpus'].head(1))[0][3000:4000])

egistrant has submitted electronically and posted on its corporate website, if any, every Interactive Data File required to be submitted and posted pursuant to Rule 405 of Regulation S-T during the preceding 12 months (or for such shorter period that the registrant was required to submit and post such files).    Yes
x
No
̈
Indicate by check mark if disclosure of delinquent filers pursuant to Item 405 of Regulation S-K is not contained herein, and will not be contained, to the best of Registrant’s knowledge, in definitive proxy or information statements incorporated by reference in Part III of this Form 10-K or any amendment to this Form 10-K.
x
Indicate by check mark whether the Registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, or a smaller reporting company. See the definitions of “large accelerated filer,” “accelerated filer” and “smaller reporting company” in Rule 12b-2 of the Exchange Act. (Check one).
Large accelerated filer
̈
Accelerated file

Lets save the file already clened, so next time we can start from here.

In [9]:
data.to_pickle('./data/10k-clean.pik')

# Extracting only MDA

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc="Processing")

data = pd.read_pickle('./data/10k-clean.pik')

In [2]:
def parsing_job(text):
    mda, end= parse_mda(text)    
    # Parse second time if first parse results in index 
    if mda and len(mda.encode('utf-8')) < 1000:
        mda, _ = parse_mda(text, start=end)
    return mda

In [3]:
def parse_mda(text, start=0):
        debug = False
        """
            Return Values
        """

        mda = ""
        end = 0

        """
            Parsing Rules
        """

        # Define start & end signal for parsing
        item7_begins = [ '\nITEM 7.', '\nITEM 7 –','\nITEM 7:', '\nITEM 7 ', '\nITEM 7\n' ]
        item7_ends   = [ '\nITEM 7A' ]
        if start != 0:
            item7_ends.append('\nITEM 7') # Case: ITEM 7A does not exist
        item8_begins = [ '\nITEM 8'  ]

        """
            Parsing code section
        """
        text = text[start:]

        # Get begin
        for item7 in item7_begins:
            begin = text.find(item7)
            if debug:
                print(item7,begin)
            if begin != -1:
                break

        if begin != -1: # Begin found
            for item7A in item7_ends:
                end = text.find(item7A, begin+1)
                if debug:
                    print(item7A,end)
                if end != -1:
                    break

            if end == -1: # ITEM 7A does not exist
                for item8 in item8_begins:
                    end = text.find(item8, begin+1)
                    if debug:
                        print(item8,end)
                    if end != -1:
                        break

            # Get MDA
            if end > begin:
                mda = text[begin:end].strip()
            else:
                end = 0

        return mda, end

In [4]:
sample_mda = data['corpus'].head(2).apply(parsing_job)
print(sample_mda[1][9000:10000])

ents. The first refinancing was in connection with the PA Acquisition. In addition, in December 2016, the Company refinanced its Senior Unsecured Notes at a lower interest rate, extending the maturity of the debt an additional 4 years to 2024.
In connection with the PA Acquisition and the Esselte Acquisition, announced in October 2016, the Company incurred
$12.8 million
of transaction and integration costs in 2016.
Foreign Currency
The strong U.S. dollar impacted our 2016 results from both a translation and transaction perspective. With respect to translation the strong U.S. dollar decreased our 2016 reported sales by
$16.9 million
, or
1%
, but slightly benefited our operating income.
26
Foreign currency translation increased operating income by
$1.6 million
, or
1%
, due to the seasonality of our Brazilian business, which earns most of its income in the fourth quarter when the Brazilian real strengthened.
With respect to the transaction impact, in response to the strengthening of the

In [5]:
data['corpus'] = data['corpus'].progress_apply(parsing_job)

Processing: 100%|██████████| 100/100 [00:00<00:00, 157.49it/s]


In [6]:
data.head()

,index,company_name,form_type,cik,date,file,corpus
0,275433,TRAVELZOO INC,10-K,1133311,2017-03-15,edgar/data/1133311/0001133311-17-000010.txt,
1,2116,ACCO BRANDS Corp,10-K,712034,2017-02-27,edgar/data/712034/0000712034-17-000012.txt,ITEM 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...
2,127781,GrowGeneration Corp.,10-K,1604868,2017-03-31,edgar/data/1604868/0001213900-17-003102.txt,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...
3,16371,Advanced Biomedical Technologies Inc.,10-K,1385799,2017-02-14,edgar/data/1385799/0001387131-17-000831.txt,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS ...
4,230264,"Primerica, Inc.",10-K,1475922,2017-02-27,edgar/data/1475922/0001564590-17-002594.txt,ITEM 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...


Lets Save to future analisys

In [7]:
data.to_json('./data/mda.json')